In [ ]:
# March 2024
# Model training - XLM-RoBERTa using pytorch
# Violeta Berdejo-Espinola

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

import numpy as np
import pandas as pd
import os
import mpu
import torch
import torch.nn as nn

print(torch.has_mps) # checking if mps is available 
device = torch.device('mps')
print(device)

import platform; 
print(platform.mac_ver()) # checking pytorch version for mac - should be arm54

True
mps
('14.4', ('', '', ''), 'arm64')


/var/folders/dz/szfgpm5s0tn_frclv0gl7m2c0000gr/T/ipykernel_99030/2082705807.py:11: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  print(torch.has_mps) # checking if mps is available


# read data

In [2]:
X_train = mpu.io.read('../data/X_train_xlm.pickle').to(device)
X_test = mpu.io.read('../data/X_test_xlm.pickle').to(device)
y_train = mpu.io.read('../data/y_train_xlm.pickle').to(device)
y_test = mpu.io.read('../data/y_test_xlm.pickle').to(device)

class_weights = mpu.io.read('../data/class_weights.pickle')

In [3]:
X_train_ros = mpu.io.read('../data/embedding_train_ros.pickle')
X_test_ros = mpu.io.read('../data/embedding_test_ros.pickle')
y_train_ros = mpu.io.read('../data/y_train_emb_ros.pickle')
y_test_ros = mpu.io.read('../data/y_test_emb_ros.pickle')

X_train_ros = torch.from_numpy(X_train_ros).to(device)
X_test_ros = torch.from_numpy(X_test_ros).to(device)
y_train_ros = torch.tensor(y_train_ros).to(device)
y_test_ros = torch.tensor(y_test_ros).to(device)

In [ ]:
len(X_train['input_ids'])
y_train.bincount()

# model training

In [4]:
from transformers import (XLMRobertaForSequenceClassification, AutoTokenizer)

id2label = {'negative': 0, 
            'positive': 1}
label2id = {0: 'negative',
            1: 'positive'}

model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base',
                                                            num_labels=2, 
                                                            id2label=id2label, 
                                                            label2id=label2id).to(device)

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

# freeze transformer model parametres

for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# creating dataloaders

from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch_size = 16

# TensorDataset(input_features, labels)

train_xlm = TensorDataset(X_train['input_ids'], X_train['attention_mask'], y_train)
train_sampler = RandomSampler(train_xlm)
train_dataloader = DataLoader(train_xlm, sampler=train_sampler, batch_size=batch_size)

test_xlm = TensorDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
test_dataloader = DataLoader(test_xlm, batch_size=batch_size)

In [ ]:
# epoch:  1 forward and backward pass of ALL training examples
# bacth size: number of training samples in one forward and backwards pass
# number of iterations: number of passes until the model has seen ALL examples. Each pass using batch_size number of samples
# if 100 samples, batch_size=20, then it does 5 iterations per epoch to see ALL examples

# if learning rate is too small:
# - model goes through many iteartions to converge
# - the loss does not change

# defining parametres for gardient descent

epochs = 8

# loss function and optimizer

loss_function = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

# total number of training steps is number of batches * number of epochs

total_steps = len(train_dataloader) * epochs

# create the learning rate scheduler

from transformers import get_linear_schedule_with_warmup

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [ ]:
# function to train model

def Trainer(model, train_dataloader, epochs, batch_size, device, optimizer, scheduler):
    # Store the average loss after each epoch 
    loss_values = []

    # number of total steps/iterations for each epoch
    print('total steps per epoch: ', len(train_dataloader) // batch_size)

    # looping over epochs
    for epoch_i in range(0, epochs):
        print('training on epoch: ', epoch_i)

        # set start time 
        t0 = time.time()
        # reset total loss
        total_loss = 0

        # model in training 
        model.train()

        # loop through batch 
        for step, batch in enumerate(train_dataloader):

            # Progress update every 50 step 
            if step % 50 == 0 and not step == 0:
                print('training on step: ', step)
                print('total time used is: {0:.2f} s'.format(time.time() - t0))

            # load data from dataloader 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # clear any previously calculated gradients 
            model.zero_grad()

            # get outputs
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            
            # get loss
            loss = outputs[0]
            
            loss.backward()
            
            # update optimizer
            optimizer.step()

            # update learning rate 
            scheduler.step()

            # total loss
            total_loss += loss.item()

            # clip the norm of the gradients to 1.0.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)         

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)
        
        print("average training loss: {0:.4f}".format(avg_train_loss))

    return loss_values

In [ ]:
%%time

torch.manual_seed(42)

import time
# results = Trainer(model, train_dataloader, epochs, batch_size, device, optimizer, scheduler)
results5  = Trainer(model, train_dataloader, epochs, batch_size, device, optimizer, scheduler)

In [ ]:
print(results3,results4)

In [ ]:
import pandas as pd

# d = pd.read_csv('../results/transformer_model_loss.csv')
df = pd.DataFrame({'2-epochs': pd.Series(results3),
                  '13-epochs': pd.Series(results4),
                  '18-epochs': pd.Series(results5)})
# df = pd.concat([d, dd], axis=1)
df
# df = df.drop(columns='Unnamed: 0').iloc[:,[0,1,2,4,3]]
print(df)
df.to_csv('../results/model_loss2.csv')

In [ ]:
# plot model loss

import matplotlib.pyplot as plt
plt.style.use('seaborn')

# epoch = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

def plot_loss(epoch, results, name):
    
    # plot
    plt.plot(epoch, results, linestyle='--',color='orange')
    # title
    plt.title('Model loss')
    # x label
    plt.xlabel('Epochs')
    # y label
    plt.ylabel('Loss')

    plt.legend(loc='best')
    return plt.savefig(fname=name, dpi=800), plt.show()

# plot
epoch = range(0,epochs)

plot_loss(epoch, results5, name='../results/Loss_epochs_18.png')

In [ ]:
# predict on the test set

import numpy as np

t0 = time.time()

def Eval(model, test_dataloader, device, ):
# model in validation mode
model.eval()

# save prediction
predictions,true_labels =[],[]

# evaluate data for one epoch
for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # validation
    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)
    
    # get output
    logits = outputs[0]
    
    # move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('mpu').numpy()
    final_prediction = np.argmax(logits, axis=-1).flatten() # predicitions with the highest probability
    
    predictions.append(final_prediction)
    true_labels.append(label_ids)
    
print('total time used is: {0:.2f} s'.format(time.time() - t0))

In [ ]:
# loss_values, outputs

In [ ]:
predictions[:3]

In [ ]:
# concatenate list of arrays

final_prediction_list = np.concatenate(predictions)
final_truelabel_list = np.concatenate(true_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cr = classification_report(final_truelabel_list, 
                           final_prediction_list)
print(cr)

In [ ]:
loss_values_2

In [ ]:
# storing loss values

loss = pd.DataFrame({'lr: 0.001': loss_values, 
                    'lr: 0.01': loss_values_2})

In [ ]:
# lineplot

import matplotlib.pyplot as plt 

plt.style.use('ggplot')

x_epoch = list(range(0, epochs))

y_loss = loss_values

plt.plot(x_epoch, y_loss, color='#881111', linestyle=':', linewidth=1, marker='.', label='learning rate: 0.001')

y_loss_2 = loss_values_2

plt.plot(x_epoch, y_loss_2, 'b--', label='learning rate: 0.01')

plt.xlabel("iterations")
plt.ylabel('loss')
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig('plot.png')

plt.show

In [ ]:
# Lastly, we might train the neural network with mini-batch (stochastic) gradient descent, which means that we split the original training dataset into 𝑀=⌈𝑁/𝐵⌉
#  batches, where 
#     𝑁 is the size of the original dataset and 
#     𝐵 is the batch size (typical numbers are 32 or 64). So, if you have, say, 𝑁=10000
#  and 𝐵=32
# , then we have 𝑀=312
#  mini-batches; so, for each epoch, you will perform 312 gradient steps, one for each mini-batch.